In [2]:
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.model_selection import train_test_split
target = 'MIS_Status'

In [3]:
! ls ./*.csv

./df_test_onehot_enc.csv  ./df_train_onehot_enc.csv  ./test.csv  ./train.csv


In [4]:
df_train = pd.read_csv("df_train_onehot_enc.csv")

In [5]:
df_ttrain, df_tval = train_test_split(df_train, train_size=0.8,random_state=1)

In [6]:
df_ttrain2 = df_ttrain.dropna()
ttrain_X = df_ttrain2.drop(target, axis=1)
ttrain_Y = df_ttrain2[target]
sm = SMOTE(random_state=0)
ttrain_x_resampled, ttrain_y_resampled = sm.fit_resample(ttrain_X, ttrain_Y)

In [12]:
import lightgbm as lgb

In [13]:
from sklearn.metrics import f1_score

In [14]:

X_test = df_tval.drop(target, axis=1)
Y_test = df_tval[target]

In [15]:
lgb_train = lgb.Dataset(ttrain_x_resampled, ttrain_y_resampled)
lgb_test = lgb.Dataset(X_test, Y_test, reference=lgb_train)

In [16]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose': -1
}

In [17]:
num_round = 1000
bst = lgb.train(params, lgb_train, num_round)

In [18]:
y_pred = bst.predict(X_test)

In [20]:
import numpy as np
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

In [21]:
f1 = f1_score(Y_test, y_pred_binary)
print(f"F1 Score: {f1}")

F1 Score: 0.33950120675784395


In [28]:
len(ttrain_X),len(ttrain_x_resampled)

(33728, 60306)

In [22]:
lgb_train = lgb.Dataset(ttrain_X, ttrain_Y)
lgb_test = lgb.Dataset(X_test, Y_test, reference=lgb_train)

In [23]:
num_round = 1000
bst = lgb.train(params, lgb_train, num_round)

In [24]:
y_pred2 = bst.predict(X_test)

In [29]:
y_pred_binary2 = np.where(y_pred2 > 0.5, 1, 0)

In [30]:
f1 = f1_score(Y_test, y_pred_binary2)
print(f"F1 Score: {f1}")

F1 Score: 0.3099752679307502
